In [2]:
import pandas as pd
import numpy as np
import re

In [3]:
dataset = pd.io.parsers.read_csv('a1_d3.txt', sep='\t',names=['data','value'])
zeroes = (dataset['value']==0).sum(axis=0)
ones = (dataset['value']==1).sum(axis=0)

# Preprocessing

In [4]:
stop = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]
dataset['data'] = dataset['data'].str.lower()
dataset['data'] = dataset['data'].str.replace(r'[-,\.&!\+\';0-9"\(\)#@$%^&?~`:'']+','',regex=True)
dataset['data'] = list(map(str.split, dataset['data']))
dataset['data'] = dataset['data'].apply(lambda x: [item for item in x if item not in stop])
# values = dataset.data.str.split(expand=True).stack()#value_counts()

In [5]:
dataset = dataset.sample(frac=1)
train_set = dataset[:int(0.8*len(dataset))]
test_set = dataset[int(0.8*len(dataset)):]
x_train = train_set['data']
y_train = train_set['value']
x_test = test_set['data']
y_test = test_set['value']
zeroes = (y_train==0).sum(axis=0)
ones = (y_train==1).sum(axis=0)
print(zeroes,ones)

411 389


In [6]:
x_train1 = x_train[y_train==1]
x_train0 = x_train[y_train==0]

In [7]:
x_train1

330                                        [comfortable]
887                                [best, headset, ever]
624    [cases, ive, tried, normally, fall, apart, mon...
247               [nice, headphones, price, work, great]
474                                     [delivery, time]
                             ...                        
72                                       [great, choice]
879    [seems, completely, secure, holding, belt, kee...
61     [really, recommend, faceplates, since, looks, ...
13                               [good, quality, though]
470             [tmobile, service, years, pretty, happy]
Name: data, Length: 389, dtype: object

In [8]:
dict = {}
count = 0
for data in x_train1:
    for item in data:
        dict[item] = 0
        count +=1
for data in x_train1:
    for item in data:
        dict[item] +=1
print("Number of words after preprocessing are",count)
dict1 = pd.DataFrame(dict.items(),columns=['word','freq'])
unique_words1 = len(dict1.index)
alpha = 1
total_words1 = count
print(np.sum(dict1['freq']))
print("Total number of unique words are ",unique_words1)
temp1 = total_words1 + unique_words1
dict1['prob'] = (dict1['freq'] + alpha)/(temp1)
print(dict1)

Number of words after preprocessing are 2048
2048
Total number of unique words are  857
            word  freq      prob
0    comfortable    10  0.003787
1           best    15  0.005508
2        headset    21  0.007573
3           ever     4  0.001721
4          cases     5  0.002065
..           ...   ...       ...
852  distracting     1  0.000688
853   completely     1  0.000688
854      keeping     1  0.000688
855   faceplates     1  0.000688
856      elegant     1  0.000688

[857 rows x 3 columns]


In [9]:
dict = {}
count = 0
for data in x_train0:
    for item in data:
        dict[item] = 0
        count +=1
for data in x_train0:
    for item in data:
            dict[item] +=1
print("Number of words after preprocessing are",count)
dict0 = pd.DataFrame(dict.items(),columns=['word','freq'])
unique_words0 = len(dict0.index)
alpha = 1
total_words0 = count
# print("Total number of unique words are ",unique_words)
temp0 = total_words0 + unique_words0
dict0['prob'] = (dict0['freq'] + alpha)/(temp0)
print(dict0)

Number of words after preprocessing are 2178
            word  freq      prob
0            get    11  0.003756
1     absolutely     1  0.000626
2       horrible     5  0.001878
3      reception     4  0.001565
4      apartment     1  0.000626
...          ...   ...       ...
1012         red     1  0.000626
1013        port     1  0.000626
1014        irda     1  0.000626
1015    pleather     1  0.000626
1016        pads     1  0.000626

[1017 rows x 3 columns]


In [10]:
x_test = pd.DataFrame(x_test,columns=['data','prob1','prob0'])
x_test['prob1'] = 0.0
x_test['prob0'] = 0.0
x_test = x_test.reset_index(inplace=False,drop=True)
i=0
for data in x_test['data']:
    for item in data:
        if item in dict1.values:
#             print(item)
            x_test['prob1'][i] += dict1.loc[dict1['word']==item,'prob']
        else:
            x_test['prob1'][i] += 1/(temp1)
    i += 1
# print(x_test)   
i=0
for data in x_test['data']:
    for item in data:
        if item in dict0.values:
#             print(item)
            x_test['prob0'][i] += dict0.loc[dict0['word']==item,'prob']
        else:
            x_test['prob0'][i] += 1/(temp0)
    i += 1
print(x_test)   
            
            

/home/badri/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/home/badri/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/home/badri/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/

                                                  data     prob1     prob0
0    [headset, works, great, packaged, nicely, avoi...  0.046127  0.008138
1         [get, extra, minutes, carry, call, get, cut]  0.006196  0.012520
2                                       [jerks, phone]  0.022031  0.020344
3    [*, comes, strong, light, use, light, camera, ...  0.017900  0.015962
4      [ive, bought, wired, headphones, sound, better]  0.021343  0.010329
..                                                 ...       ...       ...
195              [someone, shouldve, invented, sooner]  0.001377  0.001252
196  [got, aluminum, case, new, palm, vx, worked, r...  0.023752  0.016901
197                                       [good, case]  0.022375  0.007199
198         [pair, headphones, worst, ever, soundwise]  0.004131  0.008138
199                                       [kept, well]  0.007917  0.003756

[200 rows x 3 columns]


In [11]:
ans = x_test['prob1']>=x_test['prob0']
ans = ans.astype(int)
ans = np.array(ans)
y_test = np.array(y_test)

In [12]:
print(ans)
print(y_test)

[1 0 1 1 1 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 0 0 1 0 1 0 0 1 0 1 1 1 1 1 0 0 0
 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 0 1 1 1 1 0 1 0 0 1 0 1 1 1 1 0 0 0 0 0
 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 0 0 1 1 1 0 1 1 0 1 0
 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1
 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 0 1 1 1 0
 1 1 0 1 1 0 1 1 1 1 1 1 1 0 1]
[1 1 0 1 0 0 0 1 1 1 1 0 0 1 1 1 1 1 1 1 0 1 1 0 1 0 0 1 0 1 1 1 1 1 0 0 0
 1 1 1 0 1 1 1 1 0 0 0 0 0 0 1 1 0 1 1 1 0 1 0 1 0 1 1 0 1 0 1 1 0 0 0 0 0
 0 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1 1 0 1 1 0 1 0 0 0 1 1 0 0 1 1 1 0 0
 1 0 0 0 0 1 1 1 1 0 1 1 0 1 0 1 1 0 0 0 0 0 1 1 0 0 1 1 1 0 1 1 1 1 1 0 0
 0 1 0 1 1 0 0 0 1 1 0 1 0 1 0 1 0 0 1 0 0 0 0 1 1 1 0 1 0 1 0 0 0 0 1 1 0
 1 1 0 0 0 0 1 1 0 1 1 1 1 0 1]


In [13]:
print(np.sum(ans==y_test))

150


In [14]:
print(y_test.size)

200


In [17]:
y_pred = ans
y_test = y_test
true_positive = np.sum(np.logical_and((y_pred==y_test),(y_pred==1)))
true_negative = np.sum(np.logical_and((y_pred==y_test),(y_pred==0)))
false_positive = np.sum(np.logical_and((y_pred!=y_test),(y_pred==1)))
false_negative = np.sum(np.logical_and((y_pred!=y_test),(y_pred==0)))
print(true_positive,true_negative,false_positive,false_negative)
precision_1 = true_positive/(true_positive+false_positive)
precision_0 = true_negative/(true_negative+false_negative)
recall_1 = true_positive/(true_positive+false_negative)
recall_0 = true_negative/(true_negative+false_positive)
f1_score_1 = 2*(precision_1*recall_1)/(precision_1+recall_1)
f1_score_0 = 2*(precision_0*recall_0)/(precision_0+recall_0)
print(precision_1,recall_1,f1_score_1)
print(precision_0,recall_0,f1_score_0)

104 46 43 7
0.7074829931972789 0.9369369369369369 0.8062015503875969
0.8679245283018868 0.5168539325842697 0.647887323943662
